In [2]:
import os
import numpy as np
from astropy.table import Table, vstack
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
from random import shuffle
import matplotlib as mpl
import shutil

In [12]:
rotation = '1'

In [15]:
def plot_full_halo(_halo, _satids=[], _xbox_min=2.0, _d_cut=75):
    
    plot_dir = '../../plots/' 
    fig = plt.figure(figsize=(20, 20))
    ax1 = fig.add_subplot(111, projection='polar')
    #ax1.set_title(_halo + '\n' + fig_title, fontsize=25)
    table_dir = '../../../../../skysurvey/tables/merged_tables'
    table_fh = os.path.join(table_dir, _halo + '.hdf5')
    _P_table = Table.read(table_fh, format='hdf5', path='data')
    _P_table.remove_rows(np.nonzero(_P_table['Xbox']>=_xbox_min))
    t_sats = np.unique(_P_table['satids']).tolist()
    print 'list of all satids', t_sats
    for _satid in t_sats:
        if not _satid in _satids:
            #print 'removing', _satid
            _P_table.remove_rows(np.nonzero(_P_table['satids'] == _satid))

    _P_table['Rads'].name = 'Satid'
    print len(_P_table)
    ax1.scatter(_P_table['Phi'], _P_table['Satid'], s=1, alpha=.025, color='k')
    ax1.set_thetagrids(np.linspace(0.0, 360.0, 32)[:-1])
    ax1.set_theta_direction(-1)
    ax1.set_theta_zero_location("N")
    ax1.set_ylim([0, 180])
    return fig, ax1

_badwords = ['brown', 'light', 'grey', 'gray', 'white', 'ivory', 'moccasin', 'dark', 'thistle',
             'beige', 'yellow', 'lime', 'gold', 'salmon', 'khaki', 'hot', 'pink' , 'violet',
             'pale', 'olive', 'pink', 'rose', 'silver', 'peach', 'alice', 'chartreuse', 'black',
             'sea', 'seashell', 'linen', 'coral', 'fuchsia', 'sky', 'tan', 'aqua', 'lavender', 
             'snow', 'lemon', 'old', 'mint', 'cream', 'bisque', 'wheat', 'blanchedalmond','medium', 'lawn',
             'papaya', 'white', 'powder', 'lightcyan', 'slateblue', 'palevioletred', 'azure', 'honeydew', 'corn', 'gains']

_badwords *= 10
_colors = mcolors.cnames.keys()
remove = []
for bclr in _badwords:
    for clr in _colors:
        for i in range(len(clr) + 1):
            if bclr in clr[:i]:
                remove.append(clr)
_colors = [clr for clr in _colors if not clr in remove]

In [16]:
table = None
for t_fh in [fname for fname in os.listdir('./mpi') if fname.endswith('.hdf5')]:
    _table_dir = os.path.join(os.path.abspath(os.path.curdir), 'mpi')
    _table_fh = os.path.join(_table_dir, t_fh)
    if not table:
        table = Table.read(_table_fh, format='hdf5', path='data')
        #table.keep_columns(plots + ['radius'])
    else:
        new_section = Table.read(_table_fh, format='hdf5', path='data')
        #new_section.keep_columns(plots + ['radius'])
        table = vstack([table, new_section], metadata_conflicts='silent')

table.pprint(max_width=120)

radius   r0    r1   halo  xbox_cut _xbox_fixed  _xbox_mu ... sat_purity domsat domsat_mass Log10(n_stars) n_boxes rank
------ ----- ------ ---- --------- ----------- --------- ... ---------- ------ ----------- -------------- ------- ----
    24  22.8   25.2    2  0.146946         5.0  0.146946 ...   0.197964     40       8.833        5.02762     148    0
    48  45.6   50.4    2  0.436856         5.0  0.436856 ...   0.185209     34       8.323        4.41435     294    0
    48  45.6   50.4    2  0.436856         5.0  0.436856 ...   0.161648     39       8.277        3.20222      22    0
    72  68.4   75.6    2   1.01052         5.0   1.01052 ...   0.212859     19       7.194        2.57864      21    0
    96  91.2  100.8    2    1.2296         5.0    1.2296 ...   0.790679     19       7.194        2.60097      29    0
    96  91.2  100.8    2    1.2296         5.0    1.2296 ...   0.872091     20       7.405        3.99874     215    0
    96  91.2  100.8    2    1.2296         5.0  

In [ ]:
full_table = vstack([full_table, table])
full_table.write('./full_table.hdf5', path='data', overwrite=True, append=True)
full_table.pprint(max_width=130)

In [ ]:
fig = plt.figure()#figsize=(5, 5))
ax = fig.add_subplot(111, frameon=False)
ax.axis('off')
for i, clr in enumerate(_colors):
    ax.bar(0, 0, width=0, color=clr, label=str(i) + ' ' + clr)
del ax
lg = plt.legend(loc=2, ncol=4, fontsize='xx-large')
plt.show()

In [17]:

plots = ['xbox_cut', 'Log10(mu)',
         '_xbox_fixed', '_xbox_mu',
         'xbox_min',  'xbox_max', 
         'xbox_mean', 'Log10(n_stars)',
         'deg0', 'deg1', 
         'extent', 'arc_len', 
         'sat_purity', 'domsat_mass', 
         'n_segments', 'n_boxes', 
         'rank', 'domsat']

plots2 = ['xbox_cut', 'Log10(mu)', 
         'xbox_min',  'xbox_max', 
         'xbox_mean', 'Log10(n_stars)',
         'deg0', 'deg1', 
         'extent','n_segments', 
         'sat_purity','domsat',
          'n_boxes']

r0 = str(table.meta['r_start'])
r1 = str(table.meta['r_stop'])
r_stp = str(table.meta['r_step'])
r_scl = str(table.meta['r_scale'])
d0 = str(table.meta['deg_0'])[:5]
d1 = str(table.meta['deg_1'])[:5]

min_nseg = str(table.meta['min_seg_size'])
fig_title = 'r0: ' + r0  +  ' - r1: ' + r1 +  ' - r_stp: ' + r_stp +  ' - r_scl: ' + r_scl +  ' - d0: ' + d0 +  ' - d1: ' +d1 + ' - min_nseg: ' + min_nseg 


#table = Table.read('./main.hdf5', format='hdf5', path='data')
fig = plt.figure(figsize=(20, 30))
fig.suptitle(fig_title +'\n X axis = radius Kpc', fontsize=20)
nplots = len(plots)
x_axis = table['radius']
xmin = x_axis.min() - 10
xmax = x_axis.max() + 10
for plot_id, column in enumerate(plots):
    ax = fig.add_subplot((len(plots) // 2) + 1, 2, plot_id + 1)
    ax.set_ylabel(column, fontsize=20)
    y_axis = table[column]
    if column == 'extent':
        y_axis = np.rad2deg(y_axis)
    ax.scatter(x_axis, y_axis, s=10, c=table['halo'])
    ax.set_xlim([xmin, xmax])
    ax.tick_params(labelsize=16)
    ax.axes.grid()
fig.savefig('../../plots/general_rad' + rotation + '.png')
plt.close()

fig = plt.figure(figsize=(20, 25))
fig.suptitle(fig_title +'\n X axis = dom_sat mass Msun', fontsize=20)
nplots = len(plots)
x_axis = table['domsat_mass']
xmin = x_axis.min() - .5
xmax = x_axis.max() + .5
for plot_id, column in enumerate(plots2):
    ax = fig.add_subplot((len(plots) // 2) + 1, 2, plot_id + 1)
    ax.set_ylabel(column, fontsize=20)
    
    ax.scatter(x_axis, table[column], s=10, c=table['halo'])
    ax.set_xlim([xmin, xmax])
    ax.tick_params(labelsize=16)
    ax.axes.grid()
fig.savefig('../../plots/' + rotation + '_general_mass.png')
plt.close()

halo_nums = [2, 5, 7, 8, 9, 10, 12, 14, 15, 17, 20]
radii = [50, 110, 140]
offset = 5

lblsz = 20

fig = plt.figure(figsize=(20, 14))
fig.suptitle(fig_title, fontsize=20)

ax1 = fig.add_subplot(211)
ax1.set_xlim([-1, len(halo_nums)])
#ax1.set_xlabel('halo', fontsize=25)
ax1.set_xticks(range(len(halo_nums)))
ax1.set_xticklabels([str(i) for i in halo_nums])
ax1.set_ylim([0, 50])
ax1.set_ylabel('N features\n ', fontsize=25)
ax1.tick_params(labelsize=lblsz)


ax2 = fig.add_subplot(212)
ax2.set_xlim(ax1.get_xlim())
ax2.set_xlabel('halo', fontsize=25)
ax2.set_ylim([0.0, 9.0])
ax2.set_xticks(ax1.get_xticks())
ax2.set_xticklabels(ax1.get_xticklabels())
ax2.set_ylabel('log10 dom_sat mass\n ', fontsize=25)
ax2.tick_params(labelsize=lblsz)

for _i, r in enumerate(radii):
    lbl = False
    r0 = r - offset
    r1 = r + offset
    for hlo in halo_nums:
        if not lbl:
            lbl = str(r0) + ' - ' + str(r1) + ' Kpc'
        else:
            lbl = '_l'
        lims = np.nonzero(
            np.logical_and(
                np.logical_and(
                    table['radius'] >= r0,
                    table['radius'] <= r1),
                table['halo'] == hlo))
        x0 = halo_nums.index(hlo)
        x = (x0 + (0.15 * _i)) - 0.15
        ax1.vlines(x, 0, len(lims[0]), colors=_colors[_i], alpha=1, linewidth=5, label=lbl)
        
        yval = table['domsat_mass'][lims].mean()
        ax2.vlines(x, 0, yval, colors=_colors[_i], alpha=1, linewidth=5, label=lbl)
        

ax1.axes.legend(fontsize='xx-large')
ax2.axes.legend(fontsize='xx-large')
fig.savefig('../../plots/' + rotation + '_all11halos.png')
plt.close()

fig = plt.figure(figsize=(20, 7))
fig.suptitle(fig_title, fontsize=20)
ax = fig.add_subplot(111)

for r in np.unique(table['radius']):
    
    
    lims = len(table[table['radius']==r])
    
    ax.vlines(r, 0, lims, alpha=.5, linewidth=5, label=lbl)

#ax.set_xlim([0, 200])
ax.set_xlabel('\nRadius Kpc', fontsize=25)
r_strt = int(table.meta['r_start']) - 1
r_stop = int(table.meta['r_stop'])
majorrange = range(r_strt, r_stop, 20)
minorrange = [i for i in range(r_strt, r_stop, 5) if not i in majorrange]
ax.set_xticks(majorrange)
ax.set_xticks(minorrange, minor=True)
ax.set_xticklabels([str(i) for i in majorrange], rotation=90)
ax.set_xticklabels([str(i) + '-' for i in minorrange], minor=True, rotation=90)

ax.set_ylim([0, 25])
ax.set_ylabel('N features', fontsize=25)


ax.axes.tick_params(labelsize=20)
ax.axes.tick_params(labelsize=13, which='minor') 


fig.savefig('../../plots/' + rotation + '_featuresperKpc.png')
plt.close()

hlos = np.unique(table['halo'])
hlos.sort()

for halo in hlos:
    print halo

    strhalo = str(halo)
    if len(strhalo) < 2:
        strhalo = '0' + strhalo
    shalo = 'halo' + strhalo
    fh = os.path.join('../../plots/' + shalo + '_1.png')
    
    
    
    this_halo = table[table['halo'] == halo]

    satids = np.unique(this_halo['domsat'])
    print 'table', satids
    satids.sort()
    satids = satids.tolist()
    print 'list of dom satids', satids
    satclrs = mpl.cm.get_cmap('Paired', lut=len(satids))
    
    fig, ax = plot_full_halo(shalo, _satids=satids)
    
      
    this_halo.sort('radius')
    mu_line = np.unique(this_halo[this_halo['_xbox_mu'] > this_halo['_xbox_fixed']]['radius'])
    mu_line.sort()
    if not len(mu_line):
        mu_line = [0]
    
    known_sats = []
    for row in this_halo:
        
        if not len(known_sats) or not row['domsat'] in known_sats:
            known_sats.append(row['domsat'])
            lbl = str(known_sats[-1])
        else:
            lbl = '_' + lbl
        
        sat =  row['domsat']
        
        theta0 = row['deg0']
        r_extent = row['r1'] - row['r0']
        angular_extent = row['extent']
        r_start = row['r0']
        dom_satid = row['domsat']

        theta0 += (0.5 * angular_extent)

        ax.bar(theta0, r_extent, angular_extent, r_start, color=satclrs(satids.index(sat)), alpha=.15)

        ax.scatter(0, 0, marker='s', label=lbl, color=satclrs(satids.index(sat)), alpha=1)
        ax.plot(np.linspace(0, 2 * np.pi, 100), np.ones(100) * min(mu_line), linewidth=5, alpha=.01, color='r', linestyle='dashed')
    
    #leg = ax.axes.legend(fontsize='xx-large', markerscale =5)
    fig.savefig(fh)#, dpi=400)
    plt.close()
print 'done'

2
table domsat
------
    11
    19
    20
    24
    25
    31
    34
    36
    39
    40
list of dom satids [11, 19, 20, 24, 25, 31, 34, 36, 39, 40]
list of all satids [11, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 109, 111, 112]
4612584
5
table domsat
------
   116
   122
   133
   134
   135
   142
list of dom satids [116, 122, 133, 134, 135, 142]
list of all satids [116, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179

In [21]:
np.unique(table[table['halo'] == 2]['domsat'])

11
19
20
24
25
31
34
36
39
40


In [ ]:
for fh in os.listdir('../../plots'):
    if os.path.isdir(os.path.join(os.path.abspath('../../plots'),fh)):
        continue
    if fh.startswith('.'):
        continue
    rnum = fh.split('_')[0]

    newfh, ext = fh[2:].split('.')

    newfh += rnum + os.path.extsep + ext
    dirpth = os.path.abspath('../../plots')
    src = os.path.join(dirpth, fh)
    dest = os.path.join(dirpth, newfh)
    shutil.move(src, dest)

In [ ]:
_halo = 'halo10'
table_dir = '../../../../../skysurvey/tables/merged_tables/'
table_fh = os.path.join(table_dir, _halo + '.hdf5')
table = Table.read(table_fh, format='hdf5', path='data')[::100]

In [ ]:
plt.figure(figsize=(20,5))
plt.scatter(table['Rads'], table['Xbox'], s=1, cmap=plt.cm.Paired, c=table['Phi'])
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='polar')
ax.scatter(table['Phi'], table['Rads'], s=1, cmap=plt.cm.Paired, c=table['Xbox'], vmin=0, vmax =5)
ax.set_ylim([0,200])
#plt.colorbar()
plt.show()

In [28]:
_colors *= 1000
table_dir = '../../../../../skysurvey/tables/merged_tables'

for halo in [h.split('.')[0] for h in os.listdir(table_dir)]:
    hnum = int(halo[-2:])
    table_fh = os.path.join(table_dir, halo + '.hdf5')
    _table = Table.read(table_fh, format='hdf5', path='data')
    
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(111, projection='polar')
    _table = _table[np.nonzero(_table['Xbox']>=1.0)]
    satids = np.unique(_table['satids'])
    for satid in np.unique(table[table['halo'] == hnum]['domsat']):
        lims = np.nonzero(_table['satids'] == satid)
        ax.scatter(_table['Phi'][lims], _table['Rads'][lims], s=1, alpha=.025, color=_colors[satid])#, cmap=plt.cm.Paired, c=table['Xbox'], vmin=1, vmax =5)
    ax.set_ylim([0,180])
    fig.savefig('../../plots/' + halo + '_0.png')
    plt.close(fig)

In [27]:
table = Table.read('./rotation_log.hdf5', format='hdf5', path='1')

[u'indigo',
 u'firebrick',
 u'indianred',
 u'tomato',
 u'orangered',
 u'burlywood',
 u'springgreen',
 u'orange',
 u'dodgerblue',
 u'peru',
 u'chocolate',
 u'crimson',
 u'forestgreen',
 u'cyan',
 u'red',
 u'plum',
 u'turquoise',
 u'maroon',
 u'navy',
 u'magenta',
 u'blue',
 u'sienna',
 u'royalblue',
 u'green',
 u'midnightblue',
 u'teal',
 u'rebeccapurple',
 u'steelblue',
 u'orchid',
 u'cadetblue',
 u'purple']